# Подход 1: градиентный бустинг "в лоб"



# ОТЧЕТ

#### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

Данные столбцы имеют пропуски:    
first_blood_time:   0.20110048338990025      
first_blood_team:   0.20110048338990025     
first_blood_player1:   0.20110048338990025     
first_blood_player2:   0.4524015221639412     
radiant_bottle_time:   0.16138023243854777     
radiant_courier_time:   0.007117144914121156     
radiant_flying_courier_time:   0.2826185333744729     
radiant_first_ward_time:   0.018883060783708733     
dire_bottle_time:   0.1660290033940142     
dire_courier_time:   0.00695258665021084     
dire_flying_courier_time:   0.2684150982207138     
dire_first_ward_time:   0.018780211868764784   

1) в 28% случаев команда не покупает "flying_courier" в первые 5 минут игры     
2) почти в 2% случаев, команда Radiant не ставит "Ward" в первые 5 минут игры

#### 2. Как называется столбец, содержащий целевую переменную? ####
Ответ: столбец 'radiant_win'содержит целевую переменную

#### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями?  Какое качество при этом получилось? ####

1) Кросс-валидация для градиентного бустинга с 30 деревьями длится 31.34 секунды    
2) Качество составило 0.69

#### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев? ####

1) Да, имеет. Показатель метрики качества со 100 (0.71) и 150 (0.72) деревьями больше, чем с 30 (0.69).    
2) Для ускорения можно уменьшить глубину деревьев 

In [4]:
import pandas as pd
from pprint import pprint 
import time
import datetime
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier



### 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [5]:
featuresTrain = pd.read_csv('data/features.csv', index_col='match_id')
featuresTrain = featuresTrain.drop([
		'duration', 
		'tower_status_radiant', 
		'tower_status_dire', 
		'barracks_status_radiant', 
		'barracks_status_dire'
		], 
		axis=1)

### 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [6]:
def defineEmptyColumn(df):
	# count - пременная, в которой хранится кол-во пустых ячеек для каждого столбца
	count = 0
	# rowNum - переменная, в которой хранится кол-во строк в DataFrame
	rowNum = len(df.index)
	# Перебор всех столбцов
	for colName in df:
		count = df[colName].isnull().sum()
		if count > 0:
			print(colName + ":   " + repr(count/rowNum))

In [7]:
defineEmptyColumn(featuresTrain)

first_blood_time:   0.20110048338990025
first_blood_team:   0.20110048338990025
first_blood_player1:   0.20110048338990025
first_blood_player2:   0.4524015221639412
radiant_bottle_time:   0.16138023243854777
radiant_courier_time:   0.007117144914121156
radiant_flying_courier_time:   0.2826185333744729
radiant_first_ward_time:   0.018883060783708733
dire_bottle_time:   0.1660290033940142
dire_courier_time:   0.00695258665021084
dire_flying_courier_time:   0.2684150982207138
dire_first_ward_time:   0.018780211868764784


In [8]:
# 1) в 28% случаев команда не покупает "flying_courier" в первые 5 минут игры
# 2) почти в 2% случаев, команда Radiant не ставит "Ward" в первые 5 минут игры

### 3. Замените пропуски на нули с помощью функции fillna().

In [9]:
def cleanEmptyColumn(df):
	df = df.fillna(0)
	return df

### 4. Какой столбец содержит целевую переменную? Запишите его название.

Ответ: столбец 'radiant_win'содержит целевую переменную

### 5. Обучить градиентный бустинг 

In [10]:
def gradientBoosting(trainData):
	XTrain = trainData.drop('radiant_win',1)
	yTrain = trainData['radiant_win'].to_frame()

	kf = KFold(yTrain.size, n_folds=5, shuffle=True, random_state=241)
	numTrees = [10, 20, 30, 100, 150]
	for num in numTrees:
		print("Количество деревьев: ", num)
		model = GradientBoostingClassifier(n_estimators=num, random_state=241)
		startTime = datetime.datetime.now()
		scores = cross_val_score(model, XTrain, yTrain.values.ravel(), cv=kf, scoring='roc_auc', n_jobs=-1)
		print('Время обучения: ', datetime.datetime.now() - startTime)
		print(scores)
		print("_____")

In [11]:
# Замена пропусков на нули
trainData = cleanEmptyColumn(featuresTrain)
# Функция обучения градинтного бустинга
gradientBoosting(trainData)

Количество деревьев:  10
Время обучения:  0:00:13.107052
[0.66943496 0.65627754 0.66390454 0.66281223 0.66950933]
_____
Количество деревьев:  20
Время обучения:  0:00:25.062316
[0.68939011 0.67567277 0.68201026 0.67977584 0.68741889]
_____
Количество деревьев:  30
Время обучения:  0:00:40.757500
[0.69641668 0.68365441 0.68731862 0.68728458 0.69280674]
_____
Количество деревьев:  100
Время обучения:  0:01:43.702128
[0.71211028 0.70275727 0.70340522 0.70450889 0.70884943]
_____
Количество деревьев:  150
Время обучения:  0:02:32.898680
[0.71546705 0.70731614 0.70793497 0.70887821 0.71252658]
_____


# Подход 2: логистическая регрессия #

In [12]:
import pandas as pd
import numpy as np
from pprint import pprint 
import time
import datetime
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [13]:
featuresTrain = pd.read_csv('data/features.csv', index_col='match_id')

In [14]:
featuresTrain = featuresTrain.drop([
		'duration', 
		'tower_status_radiant', 
		'tower_status_dire', 
		'barracks_status_radiant', 
		'barracks_status_dire'
		], 
		axis=1)

In [15]:
def cleanEmptyColumn(df):
	# Замена всех пропусков на 0
	df = df.fillna(0)
	return df

### 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом? ###


In [16]:
def logisticRegression(XTrain, yTrain):
	scaler = StandardScaler()
	XTrain = scaler.fit_transform(XTrain)
	kf = KFold(yTrain.size, n_folds=5, shuffle=True, random_state=241)
	cRange = range(-5, 5)
	for c in cRange:
		c = 10**c
		print('Параметр С: ',c)
		model = LogisticRegression(penalty='l2', C=c, random_state=241)
		startTime = datetime.datetime.now()
		scores = cross_val_score(model, XTrain, yTrain.values.ravel(), cv=kf, scoring='roc_auc')
		print('Время обучения:', datetime.datetime.now() - startTime)
		print(scores)
		print("_____")

In [17]:
trainData = cleanEmptyColumn(featuresTrain)
XTrain = trainData.drop('radiant_win',1)
yTrain = trainData['radiant_win'].to_frame()
logisticRegression(XTrain,yTrain)

Параметр С:  1e-05
Время обучения: 0:00:03.333260
[0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
_____
Параметр С:  0.0001
Время обучения: 0:00:04.783214
[0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
_____
Параметр С:  0.001
Время обучения: 0:00:08.686889
[0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
_____
Параметр С:  0.01
Время обучения: 0:00:11.226593
[0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
_____
Параметр С:  0.1
Время обучения: 0:00:11.597037
[0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
_____
Параметр С:  1
Время обучения: 0:00:11.057722
[0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
_____
Параметр С:  10
Время обучения: 0:00:11.137336
[0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
_____
Параметр С:  100
Время обучения: 0:00:11.796987
[0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
_____
Параметр С:  1000
Время обучения: 0:00:11.444322
[0.72165734 0.71369457 0.71438593 0.71539409 0.71639938]
_

#### ОТВЕТ:     ####
С = 0.01 - лучший парамерр регуляризации   
Качество = 0.72

Это сравнимо с градиентным бустингом, где количество деревьев превышает 150   
Логистическая регрессия работает быстрее градиентного бустинга

### 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить? ###

In [18]:
def cleanCategory(X):
    X.drop('lobby_type',1)
    X.drop('r1_hero',1)
    X.drop('r2_hero',1)
    X.drop('r3_hero',1)
    X.drop('r4_hero',1)
    X.drop('r5_hero',1)
    X.drop('d1_hero',1)
    X.drop('d2_hero',1)
    X.drop('d3_hero',1)
    X.drop('d4_hero',1)
    X.drop('d5_hero',1)
    return X

In [19]:
XTrain = cleanCategory(XTrain)

In [20]:
logisticRegression(XTrain,yTrain)

Параметр С:  1e-05
Время обучения: 0:00:03.268169
[0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
_____
Параметр С:  0.0001
Время обучения: 0:00:05.336239
[0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
_____
Параметр С:  0.001
Время обучения: 0:00:08.920582
[0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
_____
Параметр С:  0.01
Время обучения: 0:00:12.840387
[0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
_____
Параметр С:  0.1
Время обучения: 0:00:11.382031
[0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
_____
Параметр С:  1
Время обучения: 0:00:11.398994
[0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
_____
Параметр С:  10
Время обучения: 0:00:10.993997
[0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
_____
Параметр С:  100
Время обучения: 0:00:11.539696
[0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
_____
Параметр С:  1000
Время обучения: 0:00:13.739834
[0.72165734 0.71369457 0.71438593 0.71539409 0.71639938]
_

#### ОТВЕТ ####
Качество после удаления категориальных признаков не изменилось      
Следовательно, категориальные признаки не влияли на кросс-валидацию 

### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts). ###

In [21]:
def numHeroes(heroes):
	print('Всего героев в игре:', len(heroes))

In [22]:
heroes = pd.read_csv('data/dictionaries/heroes.csv')
numHeroes(heroes)

Всего героев в игре: 112


### 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа. ###

In [23]:
N = heroes.shape[0]
X_pick = np.zeros((XTrain.shape[0], N))

for i, match_id in enumerate(XTrain.index):
    for p in range(5):
        X_pick[i, XTrain.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, XTrain.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1




### 5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить? ###

In [24]:
X_pick_transp = np.transpose(X_pick)
for i in range(0, N):
    XTrain["bow_"+str(i)] = pd.Series(X_pick_transp[i], index=XTrain.index)
    
logisticRegression(XTrain,yTrain)

Параметр С:  1e-05
Время обучения: 0:00:07.568966
[0.71902994 0.71120697 0.71507984 0.71291814 0.71598965]
_____
Параметр С:  0.0001
Время обучения: 0:00:10.269131
[0.74673258 0.74001936 0.74260214 0.74026026 0.74430669]
_____
Параметр С:  0.001
Время обучения: 0:00:16.300786
[0.75503591 0.74925153 0.75165321 0.74902422 0.75306145]
_____
Параметр С:  0.01
Время обучения: 0:00:20.753695
[0.75509143 0.74979078 0.75201826 0.74951065 0.75309794]
_____
Параметр С:  0.1
Время обучения: 0:00:23.257151
[0.75500689 0.74982166 0.75197248 0.74953636 0.75294704]
_____
Параметр С:  1
Время обучения: 0:00:22.984821
[0.75499645 0.74982521 0.75196506 0.74953762 0.75292372]
_____
Параметр С:  10
Время обучения: 0:00:23.247429
[0.75499654 0.74982551 0.75196402 0.74953846 0.75292095]
_____
Параметр С:  100
Время обучения: 0:00:22.968599
[0.75499658 0.74982553 0.75196381 0.74953847 0.7529206 ]
_____
Параметр С:  1000
Время обучения: 0:00:25.738178
[0.7549965  0.74982559 0.75196385 0.74953845 0.75292055]
_

#### ОТВЕТ ####

0.75. Качество увеличилось. Как оказалось, категориальных признаки имели большой вес, и стоило не удалять их а зменить на числовые аналоги

In [27]:
featuresTest = pd.read_csv('data/features_test.csv', index_col='match_id')
X_pick = np.zeros((featuresTest.shape[0], N))
for i, match_id in enumerate(featuresTest.index):
    for p in range(5):
        X_pick[i, featuresTest.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, featuresTest.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X = cleanCategory(featuresTest)

X_pick_transp = np.transpose(X_pick)
for i in range(0, N):
    X["bow_"+str(i)] = pd.Series(X_pick_transp[i], index=featuresTest.index)

X = cleanEmptyColumn(X)
scaler = StandardScaler()
XTest = scaler.fit_transform(X)


model = LogisticRegression(penalty='l2', C=0.1, random_state=241)
model.fit(XTrain,yTrain.values.ravel())

probPos = model.predict_proba(XTest)
    
print('min ' + str(min(probPos[:, 1])))
print('max ' + str(max(probPos[:, 1])))

min 0.49999999996770517
max 0.5000000000139433
